# Language Model RNN

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

os.getcwd()

'/content'

In [5]:
os.chdir("./drive/My Drive/Colab Notebooks/pytorch")

In [6]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/pytorch'

## Create Dictionary, Corpus class

In [7]:
import torch
import torch.nn as nn 
import numpy as np
from torch.nn.utils import clip_grad_norm_

In [8]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    def __len__(self):
        return len(self.word2idx)

In [9]:
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()
    
    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000 # num of words to be samples
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [13]:
# Load "Penn Treebank" dataset
corpus = Corpus()
ids = corpus.get_data('train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [19]:
ids[:, 0:2].detach()

tensor([[   0,    1],
        [  93,  718],
        [  27,  930],
        [  48, 1523],
        [ 160, 1112],
        [4434,   32],
        [  24,  801],
        [2246,   32],
        [2006,  256],
        [ 365,  477],
        [ 467, 3150],
        [5092,   24],
        [ 220,   79],
        [ 480,   24],
        [  48,   26],
        [  27,   27],
        [ 853,   39],
        [ 997,   42],
        [ 392, 5518],
        [4210,  467]])

In [20]:
## RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)

        # Forward propagate LSTM
        out, (h, c) = self.lstm(x, h)

        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))

        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [21]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

In [22]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states] # detach() will turn of backprop

In [23]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)

        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}".format(
                epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())
            ))


Epoch [1/5], Step [0/1549], Loss: 9.2078, Perplexity: 9974.35
Epoch [1/5], Step [100/1549], Loss: 6.0460, Perplexity: 422.41
Epoch [1/5], Step [200/1549], Loss: 5.9223, Perplexity: 373.27
Epoch [1/5], Step [300/1549], Loss: 5.7275, Perplexity: 307.19
Epoch [1/5], Step [400/1549], Loss: 5.6745, Perplexity: 291.35
Epoch [1/5], Step [500/1549], Loss: 5.0919, Perplexity: 162.70
Epoch [1/5], Step [600/1549], Loss: 5.1938, Perplexity: 180.15
Epoch [1/5], Step [700/1549], Loss: 5.3506, Perplexity: 210.73
Epoch [1/5], Step [800/1549], Loss: 5.1731, Perplexity: 176.46
Epoch [1/5], Step [900/1549], Loss: 5.1073, Perplexity: 165.23
Epoch [1/5], Step [1000/1549], Loss: 5.0997, Perplexity: 163.98
Epoch [1/5], Step [1100/1549], Loss: 5.3586, Perplexity: 212.43
Epoch [1/5], Step [1200/1549], Loss: 5.1714, Perplexity: 176.17
Epoch [1/5], Step [1300/1549], Loss: 5.0880, Perplexity: 162.07
Epoch [1/5], Step [1400/1549], Loss: 4.8465, Perplexity: 127.29
Epoch [1/5], Step [1500/1549], Loss: 5.1632, Perple

In [26]:
prob = torch.ones(vocab_size)
input = torch.multinomial(prob, num_samples=1).unsqueeze(1)

In [31]:
torch.multinomial(prob, num_samples=1)

tensor([2834])

In [32]:
input.fill_(0)

tensor([[0]])

In [34]:
# Test the model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set initial hidden cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))
        
        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device) # this returns [[idx]]

        for i in range(num_samples):
            # Forward propagate RNN
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sample word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print("Sampled [{}/{}] words and save to {}".format(i+1, num_samples, 'sample.txt'))

# Save the model checkpoints
torch.save(model.state_dict(), 'model.ckpt')

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt
